# 5장 회원 탈퇴를 예측

In [9]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,종일,10500,일반,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,야간,6000,일반,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,11.000000,11.0,11,11,0,2019-04-30,1


In [4]:
uselog_months.loc[uselog_months["연월"]==year_months[]]

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
2986,201804,TS989482,6
2987,201804,TS995299,4
2988,201804,TS999079,7
2989,201804,TS999231,6


In [6]:
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]  # 기준 월
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]] # 기준 월 전 월
    del tmp_before["연월"] # id기준으로 합칠라고
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)

# count_0 : 해당 연월의 count , count_1 : 해당 전 월의 count
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


# 탈퇴전월의 탈퇴고객데이터를 작성

In [7]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1] # 탈뢰 고객만 추출
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
    
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")

print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))

exit_uselog = exit_uselog.sample(frac=1).reset_index(drop=True)
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201809,PL526101,1,1.0,XXXX,C03,M,2018-06-03,2018-10-31,CA2,...,6000.0,입회비반액할인,3.000000,2.0,8.0,1.0,0.0,2018-10-31,4.0,2018-09-30 00:00:00
1,201901,AS462937,4,7.0,XXXX,C02,F,2018-12-04,2019-02-28,CA3,...,7500.0,입회비무료,5.000000,4.0,7.0,4.0,1.0,2019-02-28,2.0,2019-01-28 00:00:00
2,201808,GD001473,1,3.0,XXX,C01,M,2018-04-10,2018-09-30,CA3,...,10500.0,입회비무료,4.166667,4.0,7.0,1.0,0.0,2018-09-30,5.0,2018-08-30 00:00:00
3,201806,GD370248,6,1.0,XXX,C01,M,2017-08-01,2018-07-31,CA2,...,10500.0,입회비반액할인,3.000000,2.5,6.0,1.0,1.0,2018-07-31,11.0,2018-06-30 00:00:00
4,201901,HI205010,3,4.0,XXXXXX,C02,M,2018-08-09,2019-02-28,CA1,...,7500.0,일반,4.000000,4.0,6.0,2.0,1.0,2019-02-28,6.0,2019-01-28 00:00:00


# 지속회원 데이터 작성

In [12]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))

conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")

print(len(conti_uselog))
conti_uselog.head()

33851
27422
2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,HI562740,10,9.0,XXXX,C01,F,2018-05-06,NaN,CA2,...,종일,10500.0,입회비반액할인,7.818182,7.0,10.0,6.0,1.0,2019-04-30,11.0
1,201809,HI324110,3,3.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,종일,10500.0,일반,3.416667,3.0,7.0,1.0,1.0,2019-04-30,46.0
2,201811,HD805503,7,7.0,XXX,C01,M,2018-05-01,NaN,CA2,...,종일,10500.0,입회비반액할인,6.818182,7.0,9.0,5.0,1.0,2019-04-30,11.0
3,201805,AS183155,6,3.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,야간,6000.0,일반,4.500000,4.5,7.0,2.0,1.0,2019-04-30,47.0
4,201902,AS040841,6,4.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,종일,10500.0,일반,6.000000,6.0,10.0,4.0,1.0,2019-04-30,23.0


In [26]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True) # 지속, 탈퇴회원 데이터 합치기
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,HI562740,10,9.0,XXXX,C01,F,2018-05-06,NaN,CA2,...,10500.0,입회비반액할인,7.818182,7.0,10.0,6.0,1.0,2019-04-30,11.0,NaN
1,201809,HI324110,3,3.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,10500.0,일반,3.416667,3.0,7.0,1.0,1.0,2019-04-30,46.0,NaN
2,201811,HD805503,7,7.0,XXX,C01,M,2018-05-01,NaN,CA2,...,10500.0,입회비반액할인,6.818182,7.0,9.0,5.0,1.0,2019-04-30,11.0,NaN
3,201805,AS183155,6,3.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,6000.0,일반,4.500000,4.5,7.0,2.0,1.0,2019-04-30,47.0,NaN
4,201902,AS040841,6,4.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,10500.0,일반,6.000000,6.0,10.0,4.0,1.0,2019-04-30,23.0,NaN


# 예측할 달의 재적기간을 작성

In [27]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months) # 사용기간을 달로 표현
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,HI562740,10,9.0,XXXX,C01,F,2018-05-06,NaN,CA2,...,7.818182,7.0,10.0,6.0,1.0,2019-04-30,11.0,NaN,4,2018-10-01
1,201809,HI324110,3,3.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,3.416667,3.0,7.0,1.0,1.0,2019-04-30,46.0,NaN,39,2018-09-01
2,201811,HD805503,7,7.0,XXX,C01,M,2018-05-01,NaN,CA2,...,6.818182,7.0,9.0,5.0,1.0,2019-04-30,11.0,NaN,6,2018-11-01
3,201805,AS183155,6,3.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,4.500000,4.5,7.0,2.0,1.0,2019-04-30,47.0,NaN,36,2018-05-01
4,201902,AS040841,6,4.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,6.000000,6.0,10.0,4.0,1.0,2019-04-30,23.0,NaN,21,2019-02-01


## 결측치 제거

In [28]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               254
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [29]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2640
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2640
period                  0
now_date                0
dtype: int64

## 문자열 변수 처리

In [30]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,입회비반액할인,종일,F,9.0,1.0,4,0.0
1,일반,종일,M,3.0,1.0,39,0.0
2,입회비반액할인,종일,M,7.0,1.0,6,0.0
3,일반,야간,F,3.0,1.0,36,0.0
4,일반,종일,F,4.0,1.0,21,0.0


In [31]:
predict_data = pd.get_dummies(predict_data) # 문자열을 칼럼으로 바꿔서 숫자로 표현함 - onehot인코딩과 비슷
predict_data.sum()#head()

count_1                  19570.0
routine_flg               3209.0
period                   59370.0
is_deleted                1052.0
campaign_name_일반          2700.0
campaign_name_입회비무료        418.0
campaign_name_입회비반액할인      574.0
class_name_야간              995.0
class_name_종일             1814.0
class_name_주간              883.0
gender_F                  1727.0
gender_M                  1965.0
dtype: float64

In [ ]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

# 탈퇴회원 예측 : 의사결정나무모델

In [36]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit)) # 데이터 개수 맟춰주기 위해(오버피팅 방지)

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"] # 예측할 것
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

# 예측 결과
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.

In [40]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1213,0.0,0.0
1077,0.0,0.0
680,1.0,1.0
1265,0.0,0.0
1925,0.0,0.0


# 모델 평가 및 튜닝

In [41]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9011406844106464


In [42]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9011406844106464
0.982256020278834


In [43]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5) # 깊이를 설정함
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9220532319391636
0.9283903675538656


## 기여 변수 확인

In [44]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.372457
1,routine_flg,0.122499
2,period,0.500578
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.002096
5,class_name_종일,0.000000
6,class_name_주간,0.002369
7,gender_F,0.000000


### 탈퇴회원 예측

In [46]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
    
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [47]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.05405405 0.94594595]]
